In [1]:
!pip install bert-extractive-summarizer
!pip install spacy==2.3.7
!pip install transformers
!pip install neuralcoref
!pip install sentencepiece
!pip install tensorflow
!python -m spacy download zh_core_web_lg

  Using cached zh_core_web_lg-2.3.1-py3-none-any.whl
✔ Download and installation successful
You can now load the model via spacy.load('zh_core_web_lg')


In [2]:
!conda install pytorch torchvision torchaudio -c pytorch -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
import warnings
warnings.filterwarnings('ignore')

import spacy
import zh_core_web_lg
import neuralcoref

nlp = zh_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/7m/hsl39nr92272zml47mrxh2hc0000gn/T/jieba.cache
Loading model cost 0.446 seconds.
Prefix dict has been built successfully.


In [4]:
from summarizer import Summarizer
from summarizer.text_processors.sentence_handler import SentenceHandler
from spacy.lang.zh import Chinese
from transformers import AutoConfig, AutoTokenizer, AutoModel
import random

In [5]:
modelName = "bert-base-chinese"
modelName = "uer/roberta-base-finetuned-chinanews-chinese"
custom_config = AutoConfig.from_pretrained(modelName)
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained(modelName)
custom_model = AutoModel.from_pretrained(modelName, config=custom_config)
model = Summarizer(
    custom_model=custom_model, 
    custom_tokenizer=custom_tokenizer,
    sentence_handler = SentenceHandler(language=Chinese),
)
with open("1.txt") as file:
    plain = file.read().split("。")
    body = []
    for sentence in plain:
        if random.randint(0, 100) > 10:
            body.append(sentence)
    body = "。".join(body)

result = model(
    body=body,
    num_sentences=5
)
result

Some weights of the model checkpoint at uer/roberta-base-finetuned-chinanews-chinese were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


'一直以来总想用一种什么方式把初中生活记录下来，用文字，又太过单调，用图画，又太过死板，，只有一种叫做光影的东西才能把他们全记录的真真切切。 我们在摄像头下肆意打闹，管他见不见阎罗王，上课时在课桌下搞着小发明，幻想成为下一个爱因斯坦，翻开桌斗一看，一堆小纸条和零零散散的的橡皮泥，桌角还有用纸包得严严的口香糖。 吧啦”又一个可怜的小孩的小孩柜门被某某按进去了，随后一声尖叫，有什么办法呢，修柜吧。 曾一度有过一段医院的经历，在重症病房外要么是大喜要么是大悲，在病人命悬一线时，有了生的希望时，亲人是欣喜若狂的，他们觉得亲人活着，就已经是老天在眷顾他们了，只要有生命，已经是别无所求，与生死较量过的人，还有什么苦不能吃什么难不能过呢。 然后每日练习十二小时，平均每天击出一千球，一直练到球杆握不住为止，这就是她成功的代价。'

In [6]:
model.run_embeddings(body, ratio=0.2)

array([[ 0.00324062,  0.26196805,  0.3949575 , ...,  0.3007724 ,
         0.6028439 ,  0.3966718 ],
       [ 0.1440831 ,  0.50811267, -0.4292719 , ...,  0.29488227,
         0.10104778,  0.27318   ],
       [ 1.0714781 ,  0.01688286, -0.42002812, ...,  0.4559773 ,
        -1.0178088 , -0.11570095]], dtype=float32)

In [7]:
result

'一直以来总想用一种什么方式把初中生活记录下来，用文字，又太过单调，用图画，又太过死板，，只有一种叫做光影的东西才能把他们全记录的真真切切。 我们在摄像头下肆意打闹，管他见不见阎罗王，上课时在课桌下搞着小发明，幻想成为下一个爱因斯坦，翻开桌斗一看，一堆小纸条和零零散散的的橡皮泥，桌角还有用纸包得严严的口香糖。 吧啦”又一个可怜的小孩的小孩柜门被某某按进去了，随后一声尖叫，有什么办法呢，修柜吧。 曾一度有过一段医院的经历，在重症病房外要么是大喜要么是大悲，在病人命悬一线时，有了生的希望时，亲人是欣喜若狂的，他们觉得亲人活着，就已经是老天在眷顾他们了，只要有生命，已经是别无所求，与生死较量过的人，还有什么苦不能吃什么难不能过呢。 然后每日练习十二小时，平均每天击出一千球，一直练到球杆握不住为止，这就是她成功的代价。'

In [8]:
custom_config

BertConfig {
  "_name_or_path": "uer/roberta-base-finetuned-chinanews-chinese",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "mainland China politics",
    "1": "Hong Kong - Macau politics",
    "2": "International news",
    "3": "financial news",
    "4": "culture",
    "5": "entertainment",
    "6": "sports"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Hong Kong - Macau politics": 1,
    "International news": 2,
    "culture": 4,
    "entertainment": 5,
    "financial news": 3,
    "mainland China politics": 0,
    "sports": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "p